# Getting the imagery from Google earth engine
Google earth engine is a great resource for accessing and processing data. However, to facilitate everyone's processing workflows, Google limits the amount, types, and ways processing can occur. Moreover, to perform larger tasks on Google's servers can require purchasing additional resources. Alteratively, we can download base data and perform task off Google's services. In this notebook we will demonstrate how to stream intermediate products from Google's services for down stream analyses. Later we will use these data to estimate parameters of interest. 

In [ ]:
#import packages
import geopandas as gpd, pandas as pd, os, numpy as np
import ee, geemap

### Authenticate into Earth Engine

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jshogland') #you will want to select your personal cloud project

## Create definitions for the median and medoid procedures

In [ ]:
def maskL8sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    #Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, overwrite=True).addBands(thermalBands, overwrite=True).updateMask(qaMask).updateMask(saturationMask)

def median_mosaic(image,fltr=None,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    return inCollection.median()

def _medoid(col):
    median = ee.ImageCollection(col).median()
    diff=ee.Image(col).subtract(median).pow(ee.Image.constant(2))
    return diff.reduce('sum').addBands(col)


def medoid_mosaic(image, fltr,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    medoid = inCollection.map(_medoid)
    medoid = ee.ImageCollection(medoid).reduce(ee.Reducer.min(7)).select([1,2,3,4,5,6], refl_bands)
    return medoid



## Set various variable and create the medoid surface on ee

In [ ]:
#make lists fo band names for selections
lc8_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL']#landsat band names
tgt_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP', 'QA_PIXEL']#common band names
refl_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']#bands we care about

#specify start and end dates for the image filter
startDate = '2021-01-01'
endDate = '2024-07-01'

#Specify julian dates for filter. Here we want to select sunny months
julianStart1 = 350# Starting Julian Date (for landsat median cloud free )
julianEnd1 = 365
julianStart2 = 1
julianEnd2 = 150# Ending Julian date (for landsat median cloud free)

#define the study area extent from our convex hull
#geo=geemap.gdf_to_ee(gpd.GeoDataFrame(geometry=chul)) #convert our convex hull into a ee feature class object

#make the ee collection
l8_col=ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

#set various filters
#f_bnds=ee.Filter.bounds(geometry=geo)
f_date=ee.Filter.date(startDate,endDate)
f_cr1=ee.Filter.calendarRange(julianStart1,julianEnd1)
f_cr2=ee.Filter.calendarRange(julianStart2,julianEnd2)
f_or=ee.Filter.Or(f_cr1,f_cr2)
f_and=ee.Filter.And(f_date,f_or)

#use our filter on the landsat collection
l8=l8_col.filter(f_and).map(maskL8sr)
l8r=l8.select(lc8_bands,tgt_bands)

#call the medoid function
medoid = medoid_mosaic(l8r,fltr=f_and,refl_bands=refl_bands)

## Create definitions to iteratively download imagery from Earth Engine
This code is a little detailed and will send many requests to earth engine to accommodate their quota limits. Specifically, each chunk within the raster dataset will be a request for a image from Google earth image. So what are image chunks? Chunks are subsets of a large image that are used within Raster Tools to schedule processing. Each chunk is processed separately to accommodate parallel processing. In this case we are using dask and Raster Tools to parallelize the download of the medoid image from Google Earth image. When we save out the final image it will also get saved in parallel.

In [ ]:
import ee, geopandas as gpd, dask, xarray as xr, numpy as np, io, requests,shapely
from raster_tools import Raster, general

def _convert_array2(x,bnds,rws,clms):
    outarr=np.zeros((bnds,rws,clms),dtype='f8')
    for r in range(x.shape[0]):
        for c in range(x.shape[1]):
            vls=x[r,c]
            for b in range(bnds):
                outarr[b,r,c]=vls[b]

    return outarr

def _get_block(ee_object,ee_geo,bnds,rws,clms):
    success=True
    url=ee_object.getDownloadURL({'format':'NPY','region':ee_geo})
    try:
        #print('downloading',url)
        resp=requests.get(url)
        data=np.load(io.BytesIO(resp.content))
        #print('reformatting data')
        outarr =_convert_array2(data,bnds,rws,clms)
        #print('finished')
    except Exception as e:
        #print(e,url)
        outarr=np.zeros((bnds,rws,clms))
        success=False
    finally:
        return success,outarr,url

def _get_block_values2(ee_object,xmin,ymax,res,oprj,retry=2,block_info=None):
    bo_info=block_info[None]
    aloc=bo_info['array-location']
    bnds,rws,clms=bo_info['chunk-shape']
    xrng=aloc[2]
    yrng=aloc[1]
    xmin2=xmin+(xrng[0]*res)
    xmax2=xmin+(xrng[1]*res)
    ymax2=ymax-(yrng[0]*res)
    ymin2=ymax-(yrng[1]*res)
    ply=[
        [xmin2,ymin2],
        [xmin2,ymax2],
        [xmax2,ymax2],
        [xmax2,ymin2],
        [xmin2,ymin2]
    ]
    tbb=shapely.Polygon(ply)
    tgdf=gpd.GeoSeries(tbb,crs=oprj).buffer(-res/2.0)
    g=tgdf.to_crs('EPSG:4326').geometry.iloc[0]
    xx,yy=g.exterior.coords.xy
    x=list(xx)
    y=list(yy)
    ee_geo=ee.Geometry.Polygon(tuple(zip(x,y)))
    cnt=0
    success,outarr,url=_get_block(ee_object,ee_geo,bnds,rws,clms)
    #build a loop to get data in the case of server error
    while (success==False):
        if(cnt>retry):
            break
        print('Retry',cnt+1)
        success,outarr,url=_get_block(ee_object,ee_geo,bnds,rws,clms)
        cnt+=1

    if(success==False):
        print("Problem with url",url)

    return outarr



def get_raster(gdf, ee_object,dvs=2048):
    '''
    creates a raster dataset of specified resolution from point clouds

    gdf: geodataframe from build_extent function
    ee_object: earth engine image object
    dvs: int of number of cells of the length of a square chunk

    return: Raster of image values
    '''
    prj=ee_object.projection()
    oprj=prj.crs().getInfo()
    xmin,ymin,xmax,ymax=gdf.to_crs(oprj).total_bounds.astype('int32')
    res=prj.nominalScale().getInfo()
    xchs=int((xmax-xmin)/(dvs*res))+1
    ychs=int((ymax-ymin)/(dvs*res))+1
    xsteps=np.arange(xmin,xmin+(xchs*dvs*res),res)
    ysteps=np.arange(ymax,ymax-(ychs*dvs*res),-res)


    #make chunks
    bnds = ee_object.bandNames().getInfo()

    xchunk=tuple([dvs]*xchs)
    ychunk=tuple([dvs]*ychs)

    tda=dask.array.map_blocks(_get_block_values2,ee_object,xmin,ymax,res,oprj,chunks=((len(bnds)),ychunk,xchunk),dtype=float)
    xrs=xr.DataArray(tda,coords={'band':bnds,'y':ysteps,'x':xsteps})
    return xrs

## Get the data!

rs_medoid will just be a reference to the data but will give us the background info needed to make informed decisions.

In [ ]:
gdf=gpd.read_file('../Costa_Rica_Data/Classification_Plots.zip') #use the points to get the extent of the analysis area to download

#get medoid images
chul=gpd.GeoSeries(gdf.unary_union.convex_hull,crs=gdf.crs) #from the points create a boundary to use for downloading
rs_medoid = get_raster(chul,medoid.setDefaultProjection(crs='EPSG:5367',scale=30),256) #note the projects we are setting on Google Earth Engine
rs_medoid #show the xarray data


### Exercise 1: Understanding the procedure
- How many cells are in the image?
- How many bands are in the image?
- How many processing chunks?
- How many calls to Google Earth Engine will we be making to save out the entire image?
- How many gb will the image take up?
- If we were to download all the predictors sampled in the Costa Rica sample dataset, how many additional images would we need to download?

## Let's look at a subset of the image before downloading
### let's subset the image to 6 chunks of size (1,1776,2048)

In [ ]:
rs=Raster(rs_medoid[:,10000:12048,10000:12048]).set_crs('EPSG:5367') #convert the xarray dataset into a Raster object
rs.xdata 

### Let's process the requests and visualize the first 3 bands of the medoid data

In [ ]:
#Let's time it with Dask's progress bar
from dask.diagnostics import ProgressBar

with ProgressBar():
  brs_m=rs.load()

In [ ]:
xr.plot.imshow(brs_m.xdata[0:3,:,:],robust=True,figsize=(15,15))

### Let's save the subset image to disk and plot each of the bands separately

In [ ]:
outpath = 'medoid_subset.tif'
if(not os.path.exists(outpath)):
    brs_m.save('medoid_subset.tif')

brs_m=Raster(outpath)
brs_m.plot(x='x',y='y',col='band',col_wrap=3,figsize=(15,brs_m.nbands),cmap='PRGn',robust=True)

### Let's overlay band 1 on a interactive map

In [ ]:
brs_m.explore(band=1,robust=True,name='Medoid band 1')

### Exercise2: understanding the data 2
- What proportion of the total image did we download and visualize?
- How long did it take to download?
- How many processors did we use to download the data?
- Why did the subset image save to disk so fast when it took so long to load (what did the load function do)?

## Let's download the entire Medoid image
This can take a while and will depend on your connection speed (~30 minutes on my computer). To get an idea of the total download time you can divide the processing time from loading the subset by the proportion of the total image we downloaded. This will roughly give you the time it takes to download the entire image. 

In [ ]:
#Let's time it with Dask's progress bar
rs2=Raster(rs_medoid).set_crs('EPSG:5367') #convert xarray object to a Raster object
with ProgressBar():
  rs2.save('medoid.tif')

### Exercise 3: Downloading Google Earth Engine images
- Why are we downloading the Google Earth Engine images?
- Download each of the images you plan to use in the modelling stage of your analyses. Do you need to download all images or can you recreate some images locally?
- If two images provide identical information for your model, is it worth downloading both images?